### Pythgon for Finance: Beta Weighting your Portfolio

In this tutorial, we begin by using yfinance to import financial stock data. 

In [57]:
import pandas as pd
import datetime as dt
import numpy as np
from scipy import stats
import yfinance as yf

#### Step 1: Specify Data Range for analysis

Hew we begin by creating start and end dates using python's datetime module

In [58]:
start = dt.datetime(2021,1,1)
end = dt.datetime.now()
start, end

(datetime.datetime(2021, 1, 1, 0, 0),
 datetime.datetime(2024, 4, 16, 11, 52, 31, 924832))

#### Step 2: Select the stocks/tickers you would like to analyse

For Australian stocks, yahoo tickers require 'AX' to be specified at the end of the ticker symbol.

For other tickers, use the search bar in yahoo finance to work out other ticker structures

In [76]:
stockList = ['CBA', 'NAB', 'ANZ', 'BHP', 'WBC']
stocks = ['^AXJO'] + [i + '.AX' for i in stockList]
stocks

['^AXJO', 'CBA.AX', 'NAB.AX', 'ANZ.AX', 'BHP.AX', 'WBC.AX']

#### Step 3: call the yfinance module:

In [77]:
df = yf.download(stocks, start, end)
log_returns = np.log(df.Close/df.Close.shift(1)).dropna()
log_returns.head()

[*********************100%%**********************]  6 of 6 completed


Ticker,ANZ.AX,BHP.AX,CBA.AX,NAB.AX,WBC.AX,^AXJO
Date,,,,,,
2021-01-05,-0.003043,0.028390,-0.006228,-0.010522,-0.005619,-0.000344
2021-01-06,-0.008306,-0.002034,-0.003973,-0.012417,-0.007714,-0.011258
2021-01-07,0.037057,0.059067,0.020063,0.027725,0.031507,0.015752
2021-01-08,0.008425,-0.004916,0.012338,0.013365,0.014403,0.006815
2021-01-11,0.000419,-0.003649,-0.005504,-0.006014,0.000493,-0.009023


#### Step 4a: Directly Calculate Beta

In [61]:
def calc_beta(df):
    np_array = df.values
    # Market index is the first column 0
    m = np_array[:,0]
    beta = []
    for ind, col in enumerate(df):
        if ind > 0:
            # stock returns indexed by ind
            s = np_array[:, ind]
            # calculate covariance matrix between stock and market
            covariance = np.cov(s,m)
            beta.append(covariance[0,1]/covariance[1,1])
    return pd.Series(beta, df.columns[:-1], name='Beta')

In [62]:
calc_beta(log_returns)

Ticker
ANZ.AX    0.345308
BHP.AX    0.683814
CBA.AX    0.734904
NAB.AX    0.774700
WBC.AX    0.442070
Name: Beta, dtype: float64

#### Step 4b: Use linear regression to get coefficient of market and stocks returns

In [63]:
def regression_beta(df):
    np_array = df.values
    # Market index is the first column 0
    m = np_array[:,0]
    beta = []
    for ind, col in enumerate(df):
        if ind > 0:
            # stock returns indexed by ind
            s = np_array[:, ind]
            beta.append(stats.linregress(m,s)[0])
    return pd.Series(beta, df.columns[:-1], name='Beta')

In [64]:
regression_beta(log_returns)

Ticker
ANZ.AX    0.345308
BHP.AX    0.683814
CBA.AX    0.734904
NAB.AX    0.774700
WBC.AX    0.442070
Name: Beta, dtype: float64

#### Step 4c: Use Matrix Algebra to complete linear regression in one line

In [65]:
def matrix_beta(df):
    # Market index
    X = df.values[:, [0]]
    # add an additional column for intercept (initialize these values as 1s)
    X = np.concatenate([np.ones_like(X), X], axis=1)
    # Apply the matrix algebra for linear regression  - closed form solution
    beta = np.linalg.pinv(X.T @ X) @ X.T @ df.values[:, 1:]
    return pd.Series(beta[1], df.columns[:-1], name='Beta')

In [66]:
beta = matrix_beta(log_returns)
beta

Ticker
ANZ.AX    0.345308
BHP.AX    0.683814
CBA.AX    0.734904
NAB.AX    0.774700
WBC.AX    0.442070
Name: Beta, dtype: float64

#### Step 5: Define your portfolio and make DataFrame

Calculate Beta Weighted Portfolio

In [67]:
df.Close[-1:]

Ticker,ANZ.AX,BHP.AX,CBA.AX,NAB.AX,WBC.AX,^AXJO
Date,,,,,,
2024-04-15,29.0,45.77,114.639999,33.93,26.16,7752.5


In [68]:
units = np.array([100,250,300,400,200])
ASXprices = df.Close[-1:].values.tolist()[0]
price = np.array([round(price,2) for price in ASXprices[:-1]])
value = units*price
weight = [round(val/sum(value),2) for val in value]
beta = round(beta, 2)

In [69]:
Portfolio = pd.DataFrame({
    'Stock': stockList,
    'Direction': 'Long',
    'Type': 'S',
    'Stock Price': price,
    'Price': price,
    'Units': units,
    'Value': value,
    'Weights': weight,
    'Beta': beta,
    'Weighted Beta': weight*beta
})
Portfolio

,Stock,Direction,Type,Stock Price,Price,Units,Value,Weights,Beta,Weighted Beta
Ticker,,,,,,,,,,
ANZ.AX,CBA,Long,S,29.00,29.00,100,2900.0,0.04,0.35,0.0140
BHP.AX,NAB,Long,S,45.77,45.77,250,11442.5,0.17,0.68,0.1156
CBA.AX,ANZ,Long,S,114.64,114.64,300,34392.0,0.51,0.73,0.3723
NAB.AX,BHP,Long,S,33.93,33.93,400,13572.0,0.20,0.77,0.1540
WBC.AX,WBC,Long,S,26.16,26.16,200,5232.0,0.08,0.44,0.0352


#### Step 6: What if we have options , let's consider things in terms of Delta

In [70]:
Portfolio = Portfolio.drop(['Weighted Beta', 'Weights'], axis=1)
Portfolio['Delta'] = Portfolio['Units']
Portfolio

,Stock,Direction,Type,Stock Price,Price,Units,Value,Beta,Delta
Ticker,,,,,,,,,
ANZ.AX,CBA,Long,S,29.00,29.00,100,2900.0,0.35,100
BHP.AX,NAB,Long,S,45.77,45.77,250,11442.5,0.68,250
CBA.AX,ANZ,Long,S,114.64,114.64,300,34392.0,0.73,300
NAB.AX,BHP,Long,S,33.93,33.93,400,13572.0,0.77,400
WBC.AX,WBC,Long,S,26.16,26.16,200,5232.0,0.44,200


#### Add Options to portfolio

In [71]:
Options = [{'option': 'CBA0Z8', 'underlying': 'CBA', 'price': 3.950, 'units': 2, 'delta': 0.627, 'direction':'Short', 'type':'Call'},
          {'option': 'WPLQ89', 'underlying': 'BHP', 'price': 1.325, 'units': 2, 'delta': -0.425, 'direction':'Long', 'type':'Put'}]

In [72]:
for index, row in enumerate(Options):
    Portfolio.loc[row['option']] = [row['underlying'], row['direction'], row['type'], Portfolio.loc[row['underlying']+'.AX', 'Price'],
                                    row['price'], row['units'],row['price'] * row['units']*100, beta[row['underlying']+'.AX'],
                                   (row['delta']*row['units']*100 if row['direction'] == 'Long' else -row['delta']*row['units']*100)]
Portfolio

,Stock,Direction,Type,Stock Price,Price,Units,Value,Beta,Delta
Ticker,,,,,,,,,
ANZ.AX,CBA,Long,S,29.00,29.000,100,2900.0,0.35,100.0
BHP.AX,NAB,Long,S,45.77,45.770,250,11442.5,0.68,250.0
CBA.AX,ANZ,Long,S,114.64,114.640,300,34392.0,0.73,300.0
NAB.AX,BHP,Long,S,33.93,33.930,400,13572.0,0.77,400.0
WBC.AX,WBC,Long,S,26.16,26.160,200,5232.0,0.44,200.0
CBA0Z8,CBA,Short,Call,114.64,3.950,2,790.0,0.73,-125.4
WPLQ89,BHP,Long,Put,45.77,1.325,2,265.0,0.68,-85.0


#### Step 7: Weight the Delta's using Beta

In [73]:
Portfolio['ASX200 Weighted Delta (point)'] = round(Portfolio['Beta']*Portfolio['Stock Price']/ASXprices[0]*Portfolio['Delta'],2)
Portfolio['ASX200 Weighted Delta (1%)'] = round(Portfolio['Beta']*Portfolio['Stock Price']*Portfolio['Delta']*0.01,2)
Portfolio

,Stock,Direction,Type,Stock Price,Price,Units,Value,Beta,Delta,ASX200 Weighted Delta (point),ASX200 Weighted Delta (1%)
Ticker,,,,,,,,,,,
ANZ.AX,CBA,Long,S,29.00,29.000,100,2900.0,0.35,100.0,35.00,10.15
BHP.AX,NAB,Long,S,45.77,45.770,250,11442.5,0.68,250.0,268.31,77.81
CBA.AX,ANZ,Long,S,114.64,114.640,300,34392.0,0.73,300.0,865.73,251.06
NAB.AX,BHP,Long,S,33.93,33.930,400,13572.0,0.77,400.0,360.36,104.50
WBC.AX,WBC,Long,S,26.16,26.160,200,5232.0,0.44,200.0,79.38,23.02
CBA0Z8,CBA,Short,Call,114.64,3.950,2,790.0,0.73,-125.4,-361.87,-104.94
WPLQ89,BHP,Long,Put,45.77,1.325,2,265.0,0.68,-85.0,-91.22,-26.46


#### Step 8: Total the Delta's to get Portfolio Overview

In [78]:
Portfolio.loc['Total', ['Value', 'ASX200 Weighted Delta (point)', 'ASX200 Weighted Delta (1%)']] = Portfolio[['Value', 'ASX200 Weighted Delta (point)', 'ASX200 Weighted Delta (1%)']].sum()
Portfolio

,Stock,Direction,Type,Stock Price,Price,Units,Value,Beta,Delta,ASX200 Weighted Delta (point),ASX200 Weighted Delta (1%)
Ticker,,,,,,,,,,,
ANZ.AX,CBA,Long,S,29.00,29.000,100.0,2900.0,0.35,100.0,35.00,10.15
BHP.AX,NAB,Long,S,45.77,45.770,250.0,11442.5,0.68,250.0,268.31,77.81
CBA.AX,ANZ,Long,S,114.64,114.640,300.0,34392.0,0.73,300.0,865.73,251.06
NAB.AX,BHP,Long,S,33.93,33.930,400.0,13572.0,0.77,400.0,360.36,104.50
WBC.AX,WBC,Long,S,26.16,26.160,200.0,5232.0,0.44,200.0,79.38,23.02
CBA0Z8,CBA,Short,Call,114.64,3.950,2.0,790.0,0.73,-125.4,-361.87,-104.94
WPLQ89,BHP,Long,Put,45.77,1.325,2.0,265.0,0.68,-85.0,-91.22,-26.46
Total,NaN,NaN,NaN,NaN,NaN,NaN,68593.5,NaN,NaN,1155.69,335.14
